In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import zipfile, os

# STEP 1: Extract dataset (if not already extracted)
with zipfile.ZipFile("rice.zip", 'r') as zip_ref:
    zip_ref.extractall("rice_dataset")

# STEP 2: Setup
IMG_SIZE = (128, 128)  # smaller size = faster
BATCH_SIZE = 16
EPOCHS = 2
DATASET_DIR = "rice_dataset"

# STEP 3: Preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

class_names = list(train_gen.class_indices.keys())

# STEP 4: Build Model
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# STEP 5: Train (faster)
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[EarlyStopping(patience=1, restore_best_weights=True)]
)


In [ ]:
# STEP 6: Save
model.save("rice_model.h5")
print("✅ rice_model.h5 saved successfully!")